# Tuning

Importazione librerie

In [23]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_tuner import HyperModel, Hyperband
from kerastuner.tuners import BayesianOptimization
from sklearn.model_selection import KFold
from sklearn.utils import check_random_state

import utility as ut
from tfkan import DenseKAN

C:\Users\gobba\AppData\Local\Temp\ipykernel_27700\873042731.py:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import BayesianOptimization


Impostiamo il random state

In [24]:
# Valore del seme causale
seed_value = 0

# Impostazione dei semi casuali per os, random, numpy e tensorflow
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Controllo che il seme sia stato correttamente impostato
check_random_state(seed_value)

RandomState(MT19937) at 0x23E45C8F140

Prepariamo il dataset 

In [25]:
x_train = pd.read_csv("datasets/x_train.csv")   # Caricamento del dataset
y_train = pd.read_csv("datasets/y_train.csv")
x_test = pd.read_csv("datasets/x_test.csv")  
y_test = pd.read_csv("datasets/y_test.csv")

ut.standardize(x_train)                         # Standardizzazione
x_train = x_train.to_numpy()[:, 1:-1]
y_train = y_train.to_numpy()[:, -1]
ut.standardize(x_test)                   
x_test = x_test.to_numpy()[:, 1:-1]
y_test = y_test.to_numpy()[:, -1]

x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)   # Conversione a tensore
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)  
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


## Tuner architettura KAN

In [30]:
class HyperKAN(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
            model = tf.keras.Sequential()
            model.add(tf.keras.layers.Input(shape=self.input_shape))

            num_layers = hp.Int('num_layers', 1, 3)
            for i in range(num_layers):
                grid_range = hp.Float(f'grid_range_min_{i}', 1.0, 10.0)
                model.add(DenseKAN(
                    units=hp.Int(f'units_{i}', 1, 16),
                    grid_size=hp.Int(f'grid_size_{i}', 8, 32),
                    grid_range=[-grid_range,grid_range]
                ))

            grid_range = hp.Float(f'grid_range_min_{99}', 1.0, 10.0)
            model.add(DenseKAN(units=1,
                grid_size=hp.Int(f'grid_size_{99}', 8, 32),
                grid_range=[
                    -grid_range,
                    grid_range
            ]))

            learning_rate = hp.Float('learning_rate', 1e-3, 1, sampling='log')
            optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'adadelta', 'adagrad'])
            
            opt = tf.keras.optimizers.get(optimizer)
            opt.learning_rate = learning_rate

            model.compile(
                optimizer=opt,
                loss='mae',
                metrics=["mae", "mse"]
            )
            return model


def run_tuner(x_train, y_train, x_test, y_test, input_shape):
    hypermodel = HyperKAN(input_shape=input_shape)

    tuner = BayesianOptimization(
        hypermodel,
        objective='val_loss',
        max_trials=200,
        directory="./modelli_salvati",
        project_name='rete_kan'
    )

    kfold = KFold(n_splits=5, shuffle=True, random_state=0)

    x_train = tf.convert_to_tensor(x_train)
    y_train = tf.convert_to_tensor(y_train)

    for fold, (train_indices, val_indices) in enumerate(kfold.split(x_train.numpy())):
        print(f"Fold {fold + 1}")

        
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )

        
        x_train_fold = tf.gather(x_train, train_indices)
        x_val_fold = tf.gather(x_train, val_indices)
        y_train_fold = tf.gather(y_train, train_indices)
        y_val_fold = tf.gather(y_train, val_indices)

        tuner.search(
            x_train_fold, y_train_fold,
            epochs=100,
            validation_data=(x_val_fold, y_val_fold),
            callbacks=[early_stopping]
        )

    tuner.results_summary()
    best_model = tuner.get_best_models(num_models=1)[0]

    test_loss, test_mae = best_model.evaluate(x_test, y_test)
    print(f"Test MAE: {test_mae}")

    best_model.save('best_rete_kan_model.h5')

    return best_model

N_FEATURES = x_train.shape[1]
best_model = run_tuner(x_train, y_train, x_test, y_test, input_shape=(N_FEATURES,))

Fold 1

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_layers
3.0046            |3.0046            |grid_range_min_0
7                 |7                 |units_0
16                |16                |grid_size_0
8.6276            |8.6276            |grid_range_min_99
29                |29                |grid_size_99
0.0034426         |0.0034426         |learning_rate
adadelta          |adadelta          |optimizer

Epoch 1/100
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 4.0400 - mae: 4.0400 - mse: 21.9813 - val_loss: 3.8962 - val_mae: 3.8962 - val_mse: 20.7478
Epoch 2/100
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 3.8354 - mae: 3.8354 - mse: 20.3295 - val_loss: 3.4589 - val_mae: 3.4589 - val_mse: 17.6022
Epoch 3/100
 487/2000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3.4218 - mae: 3.4218 - mse: 17.2247

KeyboardInterrupt: 

## Tuner architettura MLP

In [34]:
class HyperKAN(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
            model = tf.keras.Sequential()
            model.add(tf.keras.layers.Input(shape=self.input_shape))

            num_layers = hp.Int('num_layers', 2, 5)
            for i in range(num_layers):
                use_reg = hp.Boolean(f'use_reg_{i}')
                model.add(tf.keras.layers.Dense(
                    units=hp.Int(f'units_{i}', 4, 256, step=4),
                    activation=hp.Choice(f'activation_{i}', ['relu', 'elu', 'selu']),
                    kernel_regularizer=tf.keras.regularizers.l2(hp.Float(f'l2_{i}', 1e-4, 1e-2, sampling='log')) if use_reg else None
                ))
            
            use_reg = hp.Boolean(f'use_reg_{i}')
            model.add(tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(hp.Float(f'l2_{i}', 1e-4, 1e-2, sampling='log')) if use_reg else None))

            learning_rate = hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')
            optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'adadelta', 'adagrad'])

            opt = tf.keras.optimizers.get(optimizer)
            opt.learning_rate = learning_rate

            model.compile(
                optimizer=opt,
                loss='mean_absolute_error',
                metrics=[tf.keras.metrics.MeanAbsoluteError(name='mae')]
            )
            return model


def run_tuner(x_train, y_train, x_test, y_test, input_shape):
    hypermodel = HyperKAN(input_shape=input_shape)

    tuner = BayesianOptimization(
        hypermodel,
        objective='val_loss',
        max_trials=100,
        directory="./modelli_salvati",
        project_name='rete_mlp',

    )


    kfold = KFold(n_splits=5, shuffle=True, random_state=0)

    x_train = tf.convert_to_tensor(x_train)
    y_train = tf.convert_to_tensor(y_train)

    for fold, (train_indices, val_indices) in enumerate(kfold.split(x_train.numpy())):
        print(f"Fold {fold + 1}")

        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
        
        x_train_fold = tf.gather(x_train, train_indices)
        x_val_fold = tf.gather(x_train, val_indices)
        y_train_fold = tf.gather(y_train, train_indices)
        y_val_fold = tf.gather(y_train, val_indices)

        tuner.search(
            x_train_fold, y_train_fold,
            epochs=50,
            validation_data=(x_val_fold, y_val_fold),
            callbacks=[early_stopping]
        )

    tuner.results_summary()
    best_model = tuner.get_best_models(num_models=1)[0]

    test_loss, test_mae = best_model.evaluate(x_test, y_test)
    print(f"Test MAE: {test_mae}")

    best_model.save('best_rete_mlp_model.h5')

    return best_model

N_FEATURES = x_train.shape[1]
best_model = run_tuner(x_train, y_train, x_test, y_test, input_shape=(N_FEATURES,))

Fold 1

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_layers
False             |False             |use_reg_0
92                |92                |units_0
relu              |relu              |activation_0
True              |True              |use_reg_1
204               |204               |units_1
selu              |selu              |activation_1
0.018077          |0.018077          |learning_rate
rmsprop           |rmsprop           |optimizer

Epoch 1/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 2s 727us/step - loss: 1.0987 - mae: 1.0887 - val_loss: 0.7967 - val_mae: 0.7894
Epoch 2/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1s 634us/step - loss: 0.8322 - mae: 0.8246 - val_loss: 0.7973 - val_mae: 0.7906
Epoch 3/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1s 628us/step - loss: 0.7998 - mae: 0.7928 - val_loss: 0.7340 - val_mae: 0.7267
Epoch 4/50
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1s 606us/step - loss: 0.7808 - mae: 0.7741 - val_loss: 0.7487 - v

KeyboardInterrupt: 